In [1]:
from __future__ import annotations

import pickle
from pathlib import Path

import numpy as np
import pandas as pd

# Explore parquet files

This notebook explores different versions of parquet files to validate the total number of events.

Gabi's old files produced with ptSkimmer:

In [ ]:
path_to_dir = Path("/eos/uscms/store/user/gmachado/bbbb/ptSkimmer/24Aug13_v12/")
year = "2023"
data_dir = Path(path_to_dir) / year

dataset = "GluGluHto2B_PT-200_M-125"

pickles_path = Path(data_dir / dataset / "pickles")
print(pickles_path)
all_pickles = list(pickles_path.glob("*.pkl"))
print("All pickle files:", all_pickles)
total_sumw = 0
for pickle_file in all_pickles:
    with Path(pickle_file).open("rb") as file:
        out_dict = pickle.load(file)
    sumw = out_dict[year][dataset]["nevents"]
    print(f"Sum of weights for {pickle_file.name}: {sumw}")
    total_sumw += sumw
print(f"Total sum of weights for all pickles for {dataset}: {total_sumw}")

/eos/uscms/store/user/kakrzyza/2024/ZLWmL/pickles
All pickle files: []
Total sum of weights for all pickles for ZLWmL: 0


In [10]:
print(Path(data_dir / dataset / "parquet").as_posix())

/eos/uscms/store/user/kakrzyza/2024/ZLWmL/parquet


In [12]:
# Load the events from the parquet files
def format_columns(columns: list):
    ret_columns = []
    for key, num_columns in columns:
        for i in range(num_columns):
            ret_columns.append(f"('{key}', '{i}')")
    return ret_columns


events_tmp = pd.read_parquet(
    Path(data_dir / dataset / "parquet").as_posix(),
    columns=format_columns([("weight", 1)]),
    filters=None,
)
events_tmp

ArrowInvalid: No match for FieldRef.Name(('weight', '0')) in __fragment_index: int32
__batch_index: int32
__last_in_fragment: bool
__filename: string

Exercise loading one sample with new parquets and pickles from the `categorizer.py` processor

In [ ]:
region = "signal-all"

In [ ]:
# define the directory structure
MAIN_DIR = "/eos/uscms/store/user/lpchbbrun3/"
dir_name = "cmantill/25Jun25_v12"
path_to_dir = Path(f"{MAIN_DIR}/{dir_name}")

year = "2022EE"
data_dir = Path(path_to_dir) / year

Let's look at all the possible samples:

In [ ]:
# list of all datasets in the directory
full_dataset_list = [p.name for p in data_dir.iterdir() if p.is_dir()]
print("Full samples list:", full_dataset_list)

In [ ]:
# Filters to apply
filters = None

# Define a single sample
# dataset = "QCD_HT-600to800"
# dataset = "JetMET_Run2022E"
dataset = "GluGluHto2B_PT-200_M-125"

# If you want to see what files are available, uncomment the following lines:
print(data_dir)
print(list(Path(data_dir / dataset / "parquet").glob(f"{region}*.parquet")))

In [ ]:
# Load one single parquet file
events_tmp = pd.read_parquet(
    # "/eos/uscms/store/user/lpchbbrun3/cmantill/25Jun25_v12/2022EE/GluGluHto2B_PT-200_M-125/parquet/signal-all_0.parquet",
    "/uscms_data/d3/cmantill/hbb/hbb-run3/signal-all_0-1.parquet",
    filters=filters,
)
events_tmp

In [ ]:
# to check one column
events_tmp["Jet2_phi"]

In [ ]:
# Load the events from the all parquet files in one folder
# Note: Adjust the filters and columns as needed

# for the signal-all region, loading all the columns is not recommended since it will take a lot of memory
# Instead, we can load only the columns we need
load_columns = ["FatJet0_msd", "weight"]

events_tmp = pd.read_parquet(
    list(Path(data_dir / dataset / "parquet").glob(f"{region}*.parquet")),
    columns=load_columns,
    filters=filters,
)
events_tmp

The following explores also the pickle files, which we need to extract the total number of MC events before a selection

In [ ]:
# Open one pickle file to load the output dictionary, this is an example
dataset = "GluGluHto2B_PT-200_M-125"
pickles_path = Path(f"{data_dir}/{dataset}/pickles/out_0.pkl")
with Path(pickles_path).open("rb") as file:
    out_dict_tmp = pickle.load(file)
out_dict_tmp

In [ ]:
# this is an example of how to access sum of genweights for a specific sample
for key in out_dict_tmp:
    print(next(iter(out_dict_tmp[key]["sumw"].values())))

In [ ]:
# Now sum for all the pickles
dataset = "QCD_HT-600to800"
# dataset = "GluGluHto2B_PT-200_M-125"

pickles_path = Path(data_dir / dataset / "pickles")
all_pickles = list(pickles_path.glob("*.pkl"))
print("All pickle files:", all_pickles)
total_sumw = 0
for pickle_file in all_pickles:
    with Path(pickle_file).open("rb") as file:
        out_dict = pickle.load(file)
    # The sum of weights is stored in the "sumw" key
    # You can access it like this:
    for key in out_dict:
        sumw = next(iter(out_dict[key]["sumw"].values()))
    print(f"Sum of weights for {pickle_file.name}: {sumw}")
    total_sumw += sumw
print(f"Total sum of weights for all pickles: {total_sumw}")

In [ ]:
# note that the weight column in the events DataFrame already accounts for:
# - the weight of each event, e.g. the generator weight, etc
# - times the luminosity
# - times the cross section of the process
events_tmp["weight_nonorm"] = events_tmp["weight"]
events_tmp["finalWeight"] = events_tmp["weight"] / total_sumw

In [ ]:
events_tmp["finalWeight"]

Let's make one dictionary that stores the sum of gen weights for all samples:

In [ ]:
samples = {
    "hbb": {"GluGluHto2B_PT-200_M-125"},
    "QCD": {
        "QCD_HT-200to400",
        "QCD_HT-400to600",
        "QCD_HT-600to800",
        "QCD_HT-800to1000",
        "QCD_HT-1000to1200",
        "QCD_HT-1200to1500",
        "QCD_HT-1500to2000",
        "QCD_HT-2000",
    },
    "TT": {"TTto2L2Nu", "TTto4Q", "TTtoLNu2Q"},
    "data": {
        "JetMET_Run2022E",
        "JetMET_Run2022F",
        "JetMET_Run2022G",
    },
    "SingleTop": {
        "TBbarQ_t-channel_4FS",
        "TbarBQ_t-channel_4FS",
        "TWminusto2L2Nu",
        "TWminusto4Q",
        "TWminustoLNu2Q",
        "TbarWplusto2L2Nu",
        "TbarWplusto4Q",
        "TbarWplustoLNu2Q",
    },
    "Diboson": {
        "WW",
        "WZ",
        "ZZ",
    },
    "WJets": {
        "Wto2Q-3Jets_HT-200to400",
        "Wto2Q-3Jets_HT-400to600",
        "Wto2Q-3Jets_HT-600to800",
        "Wto2Q-3Jets_HT-800",
    },
    "ZJets": {
        "Zto2Q-4Jets_HT-200to400",
        "Zto2Q-4Jets_HT-400to600",
        "Zto2Q-4Jets_HT-600to800",
        "Zto2Q-4Jets_HT-800",
    },
}

sum_genweights = {}
for sample_name, datasets in samples.items():
    for dataset in datasets:
        if sample_name != "data":
            # For datasets that are not data, we need to load the pickles
            # Modify the total weight column accounting for the normalization
            total_sumw = 0
            for pickle_file in list(Path(data_dir / dataset / "pickles").glob("*.pkl")):
                with Path(pickle_file).open("rb") as file:
                    out_dict = pickle.load(file)
                # The sum of weights is stored in the "sumw" key
                # You can access it like this:
                for key in out_dict:
                    sumw = next(iter(out_dict[key]["sumw"].values()))
                total_sumw += sumw

            print(f"Total sum of weights for all pickles: {total_sumw}")
            sum_genweights[dataset] = total_sumw

Now, try loading parquets for all samples:

In [ ]:
data_dir

In [ ]:
# Columns to load
load_columns = [
    "FatJet0_pt",
    "FatJet0_msd",
    "weight",
]

# These columns are specific to the signal samples
load_columns_signal = [
    "GenHPt",
    #'GenHEta',
    #'GenHPhi'
    #'GenHMass'
    #'GenHChildren',
    #'GenbEta',
    #'GenbPhi',
    #'GenbMass',
    #'GenbPt',
]

# Filters to apply
filters = None

# Load the events from the parquet files
events_dict = {}
for sample_name, datasets in samples.items():
    events_list = []
    for dataset in datasets:
        # Uncomment the following lines to see the dataset and files being loaded
        # print(f"Loading dataset: {dataset}")
        # print(f"Looking in: {data_dir / dataset / 'parquet' / f'{region}*.parquet'}")
        # print(list(Path(data_dir / dataset / "parquet").glob(f'{region}*.parquet')))

        columns_to_load = load_columns

        # Example of how to load additional columns for specific samples
        if sample_name == "hbb":
            columns_to_load = load_columns + load_columns_signal

        print(f"Loading columns: {columns_to_load}")

        events = pd.read_parquet(
            list(Path(data_dir / dataset / "parquet").glob(f"{region}*.parquet")),
            filters=filters,
            columns=columns_to_load,
        )

        if "data" not in sample_name:
            events["weight_nonorm"] = events["weight"]
            events["finalWeight"] = events["weight"] / sum_genweights.get(
                dataset, 1.0
            )  # Default to 1.0 if not found
            print(f"Using sum_genweights for {dataset}: {sum_genweights.get(dataset, 1.0)}")
        else:
            # For data, we just keep the weight as is
            events["weight_nonorm"] = events["weight"]
            events["finalWeight"] = events["weight"]

        events_list.append(events)
        print(f"Loaded {dataset: <50}: {len(events)} entries")

    # Combine all DataFrames for the process/sample
    if events_list:
        events_dict[sample_name] = pd.concat(events_list)
    else:
        print(f"No valid events loaded for sample {sample_name}.", stacklevel=1)

In [ ]:
print(events_dict.keys())

In [ ]:
events_dict["hbb"].head()

In [ ]:
events_dict["data"].head()

Let's make a simple histogram of the `FatJet_pt` feature

In [ ]:
# download the hist library: https://hist.readthedocs.io/en/latest/
import hist

# impor matplotlib for plotting
import matplotlib.pyplot as plt
import mplhep as hep

# set the style to CMS
plt.style.use(hep.style.CMS)

In [ ]:
# styles
DATA_STYLE = {
    "histtype": "errorbar",
    "color": "black",
    "markersize": 15,
    "elinewidth": 2,
    "capsize": 0,
}

# colors for the histograms
colors = {
    "hbb": "tab:purple",
    "QCD": "tab:orange",
    "TT": "tab:blue",
    "ZJets": "tab:green",
    "WJets": "tab:red",
    "SingleTop": "tab:cyan",
    "Diboson": "tab:gray",
    "data": "black",
}

# order of histograms in stack
mc_names = ["ZJets", "WJets", "Diboson", "SingleTop", "TT", "QCD"]

In [ ]:
# Define the pt histogram axes, 30 bins from 200 to 900
pt_axis = hist.axis.Regular(30, 300, 900, name="pt1", label=r"Jet $p_{T}$ [GeV]")
msd_axis = hist.axis.Regular(23, 40, 201, name="msd1", label="Jet $m_{sd}$")

# Define the sample axis
process_axis = hist.axis.StrCategory(
    [],  # This will be filled later with sample names
    name="process_sample",
    label="Sample",
    growth=True,  # Allow dynamic growth of categories
)

# Make two histograms: one normalized and one non-normalized
# The normalized histogram will be used for plotting
hist_fatjet = hist.Hist(process_axis, pt_axis, msd_axis)
hist_fatjet_nonorm = hist.Hist(process_axis, pt_axis, msd_axis)

for sample_name, events in events_dict.items():
    # Fill the histogram with weight for each sample
    hist_fatjet.fill(
        sample_name,
        events["FatJet0_pt"],
        events["FatJet0_msd"],
        weight=events["finalWeight"],
    )
    hist_fatjet_nonorm.fill(
        sample_name,
        events["FatJet0_pt"],
        events["FatJet0_msd"],
        weight=events["weight_nonorm"],  # Use non-normalized weight
    )

This is how a single histogram looks like (the 2nd axis represents the process)

In [ ]:
hist_fatjet

In [ ]:
# this selects the TT process
# and sums over the FatJet mass
hist_fatjet[{"process_sample": "TT", "msd1": sum}]

In [ ]:
# same, but this selects the QCD process
hist_fatjet[{"process_sample": "QCD", "msd1": sum}]

In [ ]:
# now let's sum over the FatJet pt to plot the mass
hist_fatjet[{"process_sample": "QCD", "pt1": sum}]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

h_tmp = hist_fatjet[{"msd1": sum}]
# h_tmp = hist_fatjet_nonorm[{"msd1": sum}]

# manually stack the histograms for each process
hep.histplot(
    [h_tmp[{"process_sample": mc_name}] for mc_name in mc_names],
    label=mc_names,
    facecolor=[colors[process_name] for process_name in mc_names],
    stack=True,
    histtype="fill",
)
ax.legend()
ax.set_yscale("log")
ax.set_ylim(1e-2, 1e9)

Now, let's add data

In [ ]:
# the error of the ratio is calculated using the Poisson interval
from hist.intervals import poisson_interval

# plot the pt
# my_data_hist = hist_fatjet[{"process_sample": "data", "msd1": sum}]
# my_mc_hists = [hist_fatjet[{"process_sample": process_name, "msd1":sum}] for process_name in mc_names]
# plot the msd
my_data_hist = hist_fatjet[{"process_sample": "data", "pt1": sum}]
my_mc_hists = [
    hist_fatjet[{"process_sample": process_name, "pt1": sum}] for process_name in mc_names
]

fig, (ax, rax) = plt.subplots(
    2, 1, figsize=(12, 12), gridspec_kw={"height_ratios": [3.5, 1], "hspace": 0.18}, sharex=True
)

# plot the data
hep.histplot(
    my_data_hist,
    ax=ax,
    histtype="errorbar",
    color="black",
    label="Data",
    capsize=4,
    yerr=True,
    flow="none",
)

# plot the MC processes
my_mc_hist_sum = sum(my_mc_hists)
hep.histplot(
    my_mc_hists,
    label=mc_names,
    facecolor=[colors[process_name] for process_name in mc_names],
    stack=True,
    histtype="fill",
    ax=ax,
)
ax.xaxis.grid(True, which="major")
ax.yaxis.grid(True, which="major")

# plot the ratio
my_ratio = my_data_hist / (my_mc_hist_sum + 1e-5)  # avoid division by zero

yerr = np.nan_to_num(
    np.abs(poisson_interval(my_data_hist.values()) - my_data_hist.values())
    / (my_mc_hist_sum.values() + 1e-5)  # avoid division by zero
)
hep.histplot(
    my_ratio,
    ax=rax,
    histtype="errorbar",
    color="black",
    label="Data/MC Ratio",
    capsize=4,
    yerr=yerr,
    flow="none",
)
rax.xaxis.grid(True, which="major")
rax.yaxis.grid(True, which="major")
ax.set_yscale("log")
ax.set_ylim(1, 4e7)
ax.legend(bbox_to_anchor=(1.03, 1), loc="upper left")

Let's load the equivalent histogram directly from the pickle file

In [ ]:
# Open one pickle file to load a histogram
pickles_path = Path(f"{data_dir}/GluGluHto2B_PT-200_M-125/pickles/out_0.pkl")
with Path(pickles_path).open("rb") as file:
    out_dict_tmp = pickle.load(file)
out_dict_tmp

In [ ]:
# sum of weights without selections
out_dict_tmp["2022EE_GluGluHto2B_PT-200_M-125"]["sumw"]["2022EE_GluGluHto2B_PT-200_M-125"]

In [ ]:
# get the histogram
out_dict_tmp["2022EE_GluGluHto2B_PT-200_M-125"]["templates"][{"region": region}]

In [ ]:
# we should compare the normalization of the histogram with the nominal weights (systematic=nominal)
out_dict_tmp["2022EE_GluGluHto2B_PT-200_M-125"]["templates"][
    {"region": region, "systematic": "nominal"}
]

In [ ]:
# from the pickle file above we see that the total sum of weights is stored in the Sum: WeightedSum(value=...) printout

# let's verify that we get the same value from the parquet file
parquet_path = Path(f"{data_dir}/GluGluHto2B_PT-200_M-125/parquet/{region}_0.parquet")
events_tmp = pd.read_parquet(parquet_path, columns=["FatJet0_pt", "FatJet0_msd", "weight"])
print("total ", events_tmp["weight"].sum())
print("w msd > 40", events_tmp.loc[events_tmp["FatJet0_msd"] > 40, "weight"].sum())

## From histograms

Here is an example of how to load the histograms from the pickle files

In [ ]:
# this will store the histograms for each sample
allhists_region = {}
# this will store the histograms for each sample without scaling by the sum of weights
allhists_region_nonorm = {}

print("region:", region)
# for sample_name in samples:
for sample_name in ["hbb"]:
    allhists_region[sample_name] = None
    for dataset in samples[sample_name]:
        print(f"Loading dataset: {dataset}, sample_name: {sample_name}")
        print(list(Path(data_dir / dataset / "pickles").glob("*.pkl")))
        for pickle_file in list(Path(data_dir / dataset / "pickles").glob("*.pkl")):
            print(f"Processing pickle file: {pickle_file}")
            with Path(pickle_file).open("rb") as file:
                out_dict = pickle.load(file)
            # there is only one key in the out_dict, so we can iterate over it
            for key in out_dict:
                # for now just take the nominal histogram
                try:
                    # we will sum over all the axes to get a single histogram of the msd
                    hist_region = out_dict[key]["templates"][{"region": region}][
                        {
                            "systematic": "nominal",
                            "dataset": key,
                            # "pnet1": sum,
                            # "mjj": sum,
                            # "genflavor": sum,
                        }
                    ]
                    print(f"Loaded histogram for {key} from {data_dir / dataset }")
                except KeyError:
                    print(f"KeyError for {region} in {data_dir / dataset / 'pickles'}, skipping.")

                sumw = sum_genweights.get(dataset, 1.0)
                print(f"Using sum_genweights for {dataset}: {sumw}")

                # if this is the first histogram, just assign it
                # otherwise, add it to the existing histogram
                if not allhists_region[sample_name]:
                    print(f"First histogram for {sample_name}, initializing.")
                    allhists_region[sample_name] = hist_region / sumw
                    allhists_region_nonorm[sample_name] = hist_region
                else:
                    print(f"Adding histogram for {sample_name}.")
                    allhists_region[sample_name] += hist_region / sumw
                    allhists_region_nonorm[sample_name] += hist_region

In [ ]:
allhists_region

In [ ]:
allhists_region["hbb"][{"pt1": sum}]

In [ ]:
allhists_region_nonorm["hbb"][{"pt1": sum}]

In [ ]:
hist_fatjet[{"process_sample": "QCD"}]

In [ ]:
allhists_region["data"]

In [ ]:
fig, (ax, rax) = plt.subplots(
    2, 1, figsize=(12, 12), gridspec_kw={"height_ratios": [3.5, 1], "hspace": 0.18}, sharex=True
)

my_data_hist = allhists_region["data"][
    {
        "pt1": sum,
        "pnet1": sum,
        "mjj": sum,
        "genflavor": sum,
    }
]
my_mc_hists = [
    allhists_region[process_name][
        {
            "pt1": sum,
            "pnet1": sum,
            "mjj": sum,
            "genflavor": sum,
        }
    ]
    for process_name in mc_names
]

# plot the data
hep.histplot(
    my_data_hist,
    ax=ax,
    histtype="errorbar",
    color="black",
    label="Data",
    capsize=4,
    yerr=True,
    flow="none",
)
# plot the MC processes
my_mc_hist_sum = sum(my_mc_hists)
hep.histplot(
    my_mc_hists,
    label=mc_names,
    facecolor=[colors[process_name] for process_name in mc_names],
    stack=True,
    histtype="fill",
    ax=ax,
)
ax.xaxis.grid(True, which="major")
ax.yaxis.grid(True, which="major")

# plot the ratio...

ax.set_yscale("log")
ax.set_ylim(1, 4e7)
ax.legend(bbox_to_anchor=(1.03, 1), loc="upper left")

The `allhists_region_nonorm` should be the same as the `hist_nonorm`

In [ ]:
allhists_region["hbb"][{"pt1": sum, "pnet1": sum, "mjj": sum, "genflavor": sum}]

In [ ]:
allhists_region_nonorm["hbb"][{"pt1": sum, "pnet1": sum, "mjj": sum, "genflavor": sum}]

In [ ]:
hist_fatjet[{"pt1": sum, "process_sample": "hbb"}]

In [ ]:
allhists_region["hbb"]